PYTHON NOTEBOOK USED TO ANSWER TO EXERCISES OF CHAPTER 4 OF MATH80624 LECTURE NOTES (Data)

Modified by:
1. Chun Peng (Created for RSOME January 2021)
2. Erick Delage (January 2021)

As discussed in Chapter 4 of the  [lecture notes](http://web.hec.ca/pages/erick.delage/MATH80624_LectureNotes.pdf) of MATH80624 at HEC Montréal. 

WARNING!!!

The following code exploits a free Mosek licence for the course MATH80624 at HEC Montréal (expiration June 1st 2021). If you have error messages informing you about licencing issues, you may try uncommenting the installation lines for Gurobi. Otherwise, we recommend that you obtain your own licence of either Mosek ([url](https://www.mosek.com/)) or Gurobi ([url](https://www.gurobi.com/)).

**Jean-Sébastien Matte**

**Sena Onen Oz**

# **Preliminaries**

In [ ]:
!pip install rsome
!pip install mosek
!rm mosek.lic
!git clone https://github.com/erickdelage/80624
!cp ./80624/mosek.lic .
!rm -r ./80624
!mkdir -p /root/mosek
!cp ./mosek.lic /root/mosek
#!pip install -i https://pypi.gurobi.com gurobipy

Cloning into '80624'...
remote: Enumerating objects: 12, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 12 (delta 2), reused 11 (delta 1), pack-reused 0
Unpacking objects: 100% (12/12), done.


In [ ]:
import rsome as rso
import numpy as np
from rsome import ro
from rsome import msk_solver as my_solver  #Import Mosek solver interface
#from rsome import grb_solver as my_solver  #Import Gurobi solver interface

### Load the data

In [ ]:
n=4    #The number of facility locations
m=12   #The number of retailer locations
c = np.array([9.1, 8.0, 4.5, 2.1])                                                        #Installation cost for each facility location
r = 2*np.ones((n,m))                                                                      #Retalier price
P = np.array([23, 168, 110, 295])                                                         #Capacity of each facility locations
D_bar = np.array([24, 12, 18, 23, 24, 13, 11, 9, 18, 25, 25, 23])                         #Nominal demand of each retalier
D_hat = np.array([18, 1, 14, 12, 13, 5, 6, 0, 4, 23, 21, 20])                             #Maximum deviation from the nominal demand
d = np.array([[2.31, 2.37, 1.89, 1.92, 1.98, 1.69, 2.37, 2.14, 2.87, 2.16, 2.15, 1.52],   #Transportation cost from facility to retailers
    [1.88, 2.36, 2.02, 2.77, 1.17, 1.45, 2.64, 1.45, 1.83, 1.80, 1.74, 2.42],
    [2.51, 1.73, 3.50, 2.39, 2.51, 2.50, 3.08, 2.36, 2.35, 1.72, 1.47, 2.10],
    [1.71, 2.99, 1.40, 0.96, 1.79, 1.81, 1.89, 2.01, 2.28, 1.71, 2.98, 2.66]])

#Column constraint generation algorithm parameters
big_M = 10000
tolerance = 1e-6

# **Nominal facility location model**

We will study the following model:
\begin{align}
\max_{x,y} & - \sum_{i=1}^n c_i x_i + \sum_{i=1}^n \sum_{j=1}^m (r_{ij}-d_{ij})y_{ij} &&\\
\text{s.t.} & \sum_{i=1}^n y_{ij} \leq D_j &&\forall\,j \in\{1,\cdots,m\}\\
& \sum_{j=1}^m y_{ij} \leq P_i x_i &&\forall\,\;i \in\{1,\cdots,n\}\\
&  x \in \{0,1\}^n,\;y_{ij} \geq 0 &&\forall\,i \in\{1,\cdots,n\}, j \in\{1,\cdots,m\}, 
\end{align}


In [ ]:
#Solve the nominal facility location model

#Create model
model=ro.Model('Nominal facility location problem')
#Define variables
x = model.dvar(n,vtype='B')
y = model.dvar((n,m))          #in million of units

#List the objective and constraints
model.max(-c@x + ((r-d)*y).sum())
model.st(y.sum(axis=0) <= D_bar)       # Maximum demand at each retalier location
model.st(y.sum(axis=1) <= P*x)         # Capacity of each facility location
model.st(y>=0)                         

#Solve the model 
model.solve(my_solver)
opt_obj = model.get()  #in million of dollars
opt_x = x.get()

print('The objective is', opt_obj, 'and the optimal faciliy location is', opt_x)

Being solved by Mosek...
Solution status: integer_optimal
Running time: 0.0211s
The objective is 89.05 and the optimal faciliy location is [1. 1. 1. 1.]


# **Exercise 4.1) Implementing vertex enumeration**

We are interested in implementing vertex enumeration on the adjustable robust optimization problem:
\begin{align}
\max_{x,y(\cdot)} \;\;& \min_{z\in\mathcal{Z}(\Gamma)}- \sum_{i=1}^n c_i x_i + \sum_{i=1}^n \sum_{j=1}^m (r_{ij}-d_{ij})y_{ij}(z) &&\\
\text{s.t.}\;\; & \sum_{i=1}^n y_{ij}(z) \leq \bar{D}_j+\hat{D}_j z_j &&\forall z\in\mathcal{Z}(\Gamma),\;\forall z\in\mathcal{Z}(\Gamma)\,,\;\forall\,j \in\{1,\cdots,m\}\\
& \sum_{j=1}^m y_{ij}(z) \leq P_i x_i &&\forall z\in\mathcal{Z}(\Gamma),\;\forall\,\;i \in\{1,\cdots,n\}\\
&  x \in \{0,1\}^n,\;y_{ij}(z) \geq 0 &&\forall z\in\mathcal{Z}(\Gamma),\;\forall\,i \in\{1,\cdots,n\}, j \in\{1,\cdots,m\}, 
\end{align}
where 
$$\mathcal{Z}:= \left\{ z\in \mathbb{R}^m\,\middle|\, -1 \leq z \leq 1,\, \sum_i |z_i| \leq \Gamma\right\}$$

In the case of $\Gamma=1$, there are $2m$ vertices to worry about. Specifically, $ \mathcal{Z}(1) = \text{ConvexHull}(\mathcal{Z}_v(1))$ with $\mathcal{Z}_v(1):=\{-e_1,-e_2,\dots,-e_m,e_1,e_2,\dots,e_m\}\;$.

In the case of $\Gamma=m$, the situation is more complicated as there are now $2^m$ vertices to take care of. Specifically, $ \mathcal{Z}(m) = \text{ConvexHull}(\mathcal{Z}_v(m))$ with $\mathcal{Z}_v(m):=\{-1,1\}^m\;$.

### Useful functions

In [ ]:
def dec2base(ns, b):   
    #This function convert a list of numbers to a 2-d np.array containing the base b representation of each of them
    def dec2base_int(n, b):    #This funcion is used to convert a digit number n to base b 
        results = []
        while True:
          if n < b:           
            results.append(n)# = [n] + results
            break
          else:
            results.append(n%b)# = [n%b] + results
            n = n//b
        results.reverse()
        return np.array(results)

    k = len(dec2base_int(max(ns),b))
    result = np.zeros((len(ns),k))
    for i in range(len(ns)):
        x = ns[i]
        tmp = dec2base_int(x, b)
        if len(tmp) < k:
            tmp = np.append(np.zeros(k-len(tmp)),tmp)
        result[i,:] = tmp
    return result

def Vertex_generation(n,Gamma):
    #This function creates an 2-d np.array containing all the vertices of an n dimensional budgeted set
    #Inputs:
    #  n: dimension of set
    #  Gamma : budget
    #Outputs:
    #  z: n \times N np.array containing N vectors representing each vertex
    
    tmp1 = dec2base(np.arange(3**n), 3)-1   #Create list of all combinations of {-1, 0, 1}^N                #convert digit to base 3 and convert a set of numbers' digit to array
    tmp2=np.sum(np.abs(tmp1),axis=1)        #Calculate amount of perturbation in each exteme point
    tsce=np.where(tmp2-Gamma==0)[0]      #find where is Gamma constraint active
    Zs=tmp1[tsce,:]
    Zs=np.transpose(Zs)
    N =np.size(Zs,axis=1)
    return (Zs, N) 


### Solution:



We reformulate the problem for vertex enumeration as 
\begin{align}
  \max_{x,\{y^k\}_{k=1}^{K}} \;\;& - \sum_{i=1}^n c_i x_i + s &&\\
  \text{s.t.}\;\; & s \leq \sum_{i=1}^n \sum_{j=1}^m (r_{ij}-d_{ij})y_{ij}^{k} && \forall \, k \in\{1, \cdots,K\} \\
  & \sum_{i=1}^n y_{ij}^{k} \leq \bar{D}_j+\hat{D}_j z_j^{k} &&\forall\,j \in\{1,\cdots,m\}, \; \forall \, k \in\{1, \cdots,K\} \\
  & \sum_{j=1}^m y_{ij}^{k} \leq P_i x_i &&\forall\,\;i \in\{1,\cdots,n\}, \; \forall \, k \in\{1, \cdots,K\}\\
  & y_{ij}^{k} \geq 0 && \forall\,i \in\{1,\cdots,n\}, j \in\{1,\cdots,m\}, \; \forall \, k \in\{1, \cdots,K\} \\
  &  x \in \{0,1\}^n, 
\end{align}
where the $z^{k}$ correspond to each of the vertices generated from function Vexter_generation.

$\Gamma=1$

In [ ]:
#Budget of uncertainty
Gamma = 1
(zbar,size_z) = Vertex_generation(m,Gamma)

#Create model
model_v=ro.Model('Vertex Enumeration-Gamma1')
#Define variables
x = model_v.dvar(n,vtype='B')
s = model_v.dvar()
#List the objective and constraints
for i in range(size_z):
  y = model_v.dvar((n,m))          #in million of units
  model_v.st(y.sum(axis=0) <= D_bar+D_hat*zbar[:,i])       # Maximum demand at each retalier location
  model_v.st(s<=((r-d)*y).sum())
  model_v.st(y.sum(axis=1) <= P*x)         # Capacity of each facility location
  model_v.st(y>=0)

model_v.max(-c@x + s)
#Solve the model 
model_v.solve(my_solver)


opt_obj_41_G1 = model_v.get()  # include optimal value here
opt_x_41_G1 = x.get()
 # include solution here
print('when Gamma=', Gamma, 'the objective is', np.round(opt_obj_41_G1,decimals=4), 'and the optimal facility locations are',opt_x_41_G1)

Being solved by Mosek...
Solution status: integer_optimal
Running time: 0.0443s
when Gamma= 1 the objective is 76.57 and the optimal facility locations are [1. 1. 1. 1.]


$\Gamma=m$


In [ ]:
#Budget of uncertainty
Gamma = m
(zbar,size_z) = Vertex_generation(m,Gamma)

In [ ]:


#Create model
model_vm=ro.Model('Vertex Enumeration-Gamma1')
#Define variables
x = model_vm.dvar(n,vtype='B')
s = model_vm.dvar()
#List the objective and constraints
y = model_vm.dvar((n,m,size_z))          #in million of units
for i in range(size_z):

  model_vm.st(y[:,:,i].sum(axis=0) <= D_bar+D_hat*zbar[:,i])       # Maximum demand at each retalier location
  model_vm.st(s<=((r-d)*y[:,:,i]).sum())
  model_vm.st(y[:,:,i].sum(axis=1) <= P*x)         # Capacity of each facility location
model_vm.st(y>=0)

model_vm.max(-c@x + s)
#Solve the model 
model_vm.solve(my_solver)

opt_obj_41_Gm = model_vm.get()  # include optimal value here
opt_x_41_Gm = x.get() # include solution here
print('when Gamma=', Gamma, 'the objective is', np.round(opt_obj_41_Gm,decimals=4), 'and the optimal facility locations are',opt_x_41_Gm)

Being solved by Mosek...
Solution status: integer_optimal
Running time: 15.7944s
when Gamma= 12 the objective is 28.51 and the optimal facility locations are [0. 1. 0. 1.]


# **Exercise 4.3) Implementing column-and-constraint generation**

Solve the robust two-stage optimization problem presented in problem (4.18) using column-and-constraint generation for the budgeted uncertainty set when $\Gamma=4$. 

### Solution:


We reformulate the problem as a master and a slave problem to implement column&constraint generation. 

The master problem is:
\begin{align}
  \max_{x,\{y^k\}_{k=1}^{K'}} \;\;& - \sum_{i=1}^n c_i x_i + s &&\\
  \text{s.t.}\;\; & s \leq \sum_{i=1}^n \sum_{j=1}^m (r_{ij}-d_{ij})y_{ij}^{k} && \forall \, k \in\{1, \cdots,K'\} \\
  & \sum_{i=1}^n y_{ij}^{k} \leq \bar{D}_j+\hat{D}_j z_j^{k} &&\forall\,j \in\{1,\cdots,m\}, \; \forall \, k \in\{1, \cdots,K'\} \\
  & \sum_{j=1}^m y_{ij}^{k} \leq P_i x_i &&\forall\,\;i \in\{1,\cdots,n\}, \; \forall \, k \in\{1, \cdots,K'\}\\
  & y_{ij}^{k} \geq 0 && \forall\,i \in\{1,\cdots,n\}, j \in\{1,\cdots,m\}, \; \forall \, k \in\{1, \cdots,K'\} \\
  &  x \in \{0,1\}^n, 
\end{align}

where $z^{k}$ are a subset of $K' < K$ vertices generated by the Vertex_generation function. 

The slave problem is: 
\begin{align}
  \min_{x, y, \lambda, \gamma, \theta, u, v, w, z} & \; -\sum_{i=1}^n c_i x_i + \sum_{i=1}^n \sum_{j=1}^m (r_{ij}-d_{ij})y_{ij} & \\
  \text{s.t.}\;\; & \sum_{i=1}^n y_{ij} \leq \bar{D}_j+\hat{D}_j z_j & \forall\,j \in\{1,\cdots,m\} \\
  & \sum_{j=1}^m y_{ij} \leq P_i x_i &\forall\,\;i \in\{1,\cdots,n\} \\
  & -y_{ij} \leq 0 & \forall\,i \in\{1,\cdots,n\}, j \in\{1,\cdots,m\} \\
  & x \in \{0,1\}^n \\
  & \lambda_{j} + \gamma_{i} - \theta_{ij} = r_{ij}-d_{ij} & \forall i \in\{1,\cdots,n\}, \; \forall j \in\{1,\cdots,m\} \\
  & \lambda, \gamma, \theta \geq 0 \\
  & \lambda_{j} \leq Mu_{j} & \forall j \in\{1,\cdots,m\} \\
  & \gamma_{i} \leq Mv_{i} & \forall i \in\{1,\cdots,n\} \\
  & \theta_{ij} \leq Mw_{ij} & \forall i \in\{1,\cdots,n\}, \; \forall j \in\{1,\cdots,m\} \\
  & \bar{D}_j+\hat{D}_j z_j - \sum_{i=1}^n y_{ij} \leq M(1-u_{j}) & \forall j \in\{1,\cdots,m\} \\
  & P_i x_i - \sum_{j=1}^m y_{ij} \leq M(1-v_{i}) & \forall i \in\{1,\cdots,n\} \\
  & y_{ij} \leq M(1-w_{ij}) & \forall i \in\{1,\cdots,n\}, \; \forall j \in\{1,\cdots,m\} \\
  & |z| \leq 1 \\
  & \sum_i |z_i| \leq \Gamma
\end{align}



In [ ]:
#Budget of uncertainty
Gamma_4 = 4
(zbar_4,size_z_4) = Vertex_generation(m,Gamma_4)
# print(zbar.size)  # 95,040

# Define the MASTER model
def master(m_z):
  master_G4 = ro.Model('master_G4')

  #Define variables
  x_mG4 = master_G4.dvar(n, vtype='B')
  s_mG4 = master_G4.dvar(1)

  # Constraints
  for i in range(m_z.shape[1]):
    y_mG4 = master_G4.dvar((n, m))          #in million of units 
    master_G4.st(s_mG4 <= ((r-d)*y_mG4).sum())
    master_G4.st(y_mG4.sum(axis=0) <= D_bar + (np.diag(D_hat) @ m_z[:,i])) # Maximum demand at each retalier location, for each vertex
    master_G4.st(y_mG4.sum(axis=1) <= P * x_mG4)         # Capacity of each facility location
    master_G4.st(y_mG4 >= 0) 

  # objective function
  master_G4.max(-c@x_mG4 + s_mG4)                        

  #Solve the model 
  master_G4.solve(my_solver)

  return master_G4.get(), x_mG4.get()

def slave(s_x):
  slave_G4 = ro.Model('slave_G4')

  # Define variables
  z_sG4 = slave_G4.dvar(m)
  y_sG4 = slave_G4.dvar((n, m))
  lambda_sG4 = slave_G4.dvar(m)
  u_sG4 = slave_G4.dvar(m, vtype='B')
  gamma_sG4 = slave_G4.dvar(n)
  v_sG4 = slave_G4.dvar(n, vtype='B')
  theta_sG4 = slave_G4.dvar((n,m))
  w_sG4 = slave_G4.dvar((n,m), vtype='B')

  # objective function
  slave_G4.min(-c@s_x + ((r-d)*y_sG4).sum())

  # Constraints
  slave_G4.st(y_sG4.sum(axis=0) <= D_bar + (np.diag(D_hat) @ z_sG4)) 
  slave_G4.st(y_sG4.sum(axis=1) <= P * s_x)   
  slave_G4.st(-y_sG4 <= 0)  
  for j in range(y_sG4.shape[0]):
    slave_G4.st(lambda_sG4 + (np.ones(m) * gamma_sG4[j]) - theta_sG4[j,:] == (r[j,:] - d[j,:]))
  slave_G4.st(lambda_sG4 >= 0) 
  slave_G4.st(gamma_sG4 >= 0)
  slave_G4.st(theta_sG4 >= 0)
  slave_G4.st(lambda_sG4 <= big_M*u_sG4) 
  slave_G4.st(gamma_sG4 <= big_M*v_sG4)
  slave_G4.st(theta_sG4 <= big_M*w_sG4)
  slave_G4.st(D_bar + (np.diag(D_hat) @ z_sG4) - y_sG4.sum(axis=0) <= big_M*(np.ones(m)-u_sG4)) 
  slave_G4.st((P * s_x) - y_sG4.sum(axis=1) <= big_M*(np.ones(n)-v_sG4))   
  slave_G4.st(y_sG4 <= big_M*(np.ones((n,m))-w_sG4)) 
  slave_G4.st(abs(z_sG4) <= 1)
  slave_G4.st(rso.norm(z_sG4, 1) <= Gamma_4)

  #Solve the model 
  slave_G4.solve(my_solver)

  return slave_G4.get(), z_sG4.get()


# C&CG Algorithm
x0 = [0, 0, 0, 0]

print('Solving the slave problem for x0')
print('')
_, z0 = slave(x0)

Z_prime = np.array([[i] for i in z0])

print('Iterating over the Constraint&Column Generation Algorithm')
print('')
for iter in range(10):
  print('iteration ', iter)
  s_hat, x_hat = master(Z_prime)

  h_hat, z_hat = slave(x_hat)

  if abs(h_hat - s_hat) <= tolerance:
    opt_obj_43 = h_hat 
    opt_x_43 = x_hat 
    print('when Gamma=', Gamma_4, 'the objective is', np.round(opt_obj_43,decimals=4), 'and the optimal facility locations are',opt_x_43)
    break
  
  else:
    Z_prime = np.column_stack((Z_prime, np.array(z_hat)))
    print('')

Solving the slave problem for x0

Being solved by Mosek...
Solution status: integer_optimal
Running time: 0.0093s
Iterating over the Constraint&Column Generation Algorithm

iteration  0
Being solved by Mosek...
Solution status: integer_optimal
Running time: 0.0122s
Being solved by Mosek...
Solution status: integer_optimal
Running time: 0.0113s

iteration  1
Being solved by Mosek...
Solution status: integer_optimal
Running time: 0.0127s
Being solved by Mosek...
Solution status: integer_optimal
Running time: 0.0097s

iteration  2
Being solved by Mosek...
Solution status: integer_optimal
Running time: 0.0154s
Being solved by Mosek...
Solution status: integer_optimal
Running time: 0.0100s

iteration  3
Being solved by Mosek...
Solution status: integer_optimal
Running time: 0.0238s
Being solved by Mosek...
Solution status: integer_optimal
Running time: 0.0120s

iteration  4
Being solved by Mosek...
Solution status: integer_optimal
Running time: 0.0206s
Being solved by Mosek...
Solution stat

In [ ]:
# As a comparison, we run the vertex enumeration algorithm on Gamma = 4 problem
#Budget of uncertainty
Gamma = 4
(zbar,size_z) = Vertex_generation(m,Gamma)

#Create model
model_vm=ro.Model('Vertex Enumeration-Gamma1')
#Define variables
x = model_vm.dvar(n,vtype='B')
s = model_vm.dvar()
#List the objective and constraints
for i in range(size_z):
  y = model_vm.dvar((n,m))          #in million of units
  model_vm.st(y.sum(axis=0) <= D_bar+D_hat*zbar[:,i])       # Maximum demand at each retalier location
  model_vm.st(s<=((r-d)*y).sum())
  model_vm.st(y.sum(axis=1) <= P*x)         # Capacity of each facility location
  model_vm.st(y>=0)

model_vm.max(-c@x + s)
#Solve the model 
model_vm.solve(my_solver)

opt_obj_43_4 = model_vm.get()  # include optimal value here
opt_x_43_4 = x.get() # include solution here
print('when Gamma=', Gamma, 'the objective is', np.round(opt_obj_43_4,decimals=4), 'and the optimal facility locations are',opt_x_43_4)

KeyboardInterrupt: ignored